In [1]:
pip install dash plotly pandas

Note: you may need to restart the kernel to use updated packages.


## Raport interactiv realizat cu Python si Plotly

In acest proiect am utilizat limbajul Python impreuna cu biblioteca Plotly pentru a construi grafice interactive pe baza setului de date analizat. Scopul a fost explorarea relatiilor dintre buget, venit, profit si ROI (return on investment), precum si evidentierea distributiei acestor valori.

Graficele de tip bar pentru top filme dupa ROI arata ca filmele cu cele mai mari castiguri raportate la buget nu sunt neaparat cele cu bugete mari. Exista filme cu buget relativ mic care obtin un ROI foarte ridicat, ceea ce indica o eficienta mai mare a investitiei. In acelasi timp, unele filme cu bugete foarte mari au un ROI scazut, ceea ce sugereaza ca un buget mare nu garanteaza automat succesul financiar.

Utilizarea axei logaritmice pe axa Y este necesara deoarece valorile ROI, bugetului si profitului variaza foarte mult. Fara aceasta scalare, diferentele dintre valori mici si valori foarte mari nu ar putea fi observate clar. Axa logaritmica nu modifica datele, ci doar modul in care sunt afisate, permitand comparatii vizuale mai corecte intre filme.

De asemenea, se observa ca distributia valorilor este puternic dezechilibrata, cu multe filme concentrate in zona valorilor mici si cateva valori extreme. Graficele interactive Plotly permit inspectarea exacta a valorilor prin hover si ofera o intelegere mai clara a structurii setului de date.




In [9]:
# TEMA 3 (dupa-feedback) - Raport interactiv Plotly/Dash: ROI (rentabilitate) in loc de profit absolut
import ast
import numpy as np
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html

# 0) Citire dataset
df = pd.read_excel("cleaned_movies_metadata.xlsx")

# 1) Conversii numerice
for col in ["budget", "revenue", "popularity", "vote_average"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# 2) Profit + ROI (rentabilitate)
df["profit"] = df["revenue"] - df["budget"]

# ROI = profit / budget; evitam impartirea la 0 si bugete lipsa
df["roi"] = np.where(df["budget"] > 0, df["profit"] / df["budget"], np.nan)

# Curatare minimala pentru graficele ROI
df_roi = df.dropna(subset=["roi", "vote_average", "title"]).copy()

# Popularity pentru marimea bulinelor (fara NaN)
if "popularity" in df_roi.columns:
    df_roi["popularity_size"] = df_roi["popularity"].fillna(0)
else:
    df_roi["popularity_size"] = 0



# =========================
# GRAFIC 1: Top 10 filme dupa ROI
# =========================
top_roi = df_roi.sort_values(by="roi", ascending=False).head(10)

fig1 = px.bar(
    top_roi,
    x="title",
    y="roi",
    title="Top 10 filme dupa ROI (profit/buget)",
    labels={"roi": "ROI (profit / buget)"}
    
)
fig1.update_yaxes(type="log", dtick=1) # Scala logaritmica pe Y pentru vizibilitate mai buna, doar puteri ale lui 10  


# =========================
# Grafic 2 : ROI mediu in functie de rating
# Agregam datele pentru a evita aglomerarea punctelor
# =========================

roi_by_rating = (
    df_roi
    .groupby('vote_average')
    .agg(
        avg_roi=('roi', 'mean'),
        count=('title', 'count')
    )
    .reset_index()
)

fig2 = px.scatter(
    roi_by_rating,
    x='vote_average',
    y='avg_roi',
    size='count',
    size_max=30,
    title='ROI mediu in functie de rating (dimensiune = numar filme)',
    labels={
        'vote_average': 'Rating',
        'avg_roi': 'ROI mediu'
    }
)

fig2.update_yaxes(
    type="log",
    range=[0, 7]   # 10^0 -> 10^7
)

fig2.update_traces(
    marker=dict(
        sizemin=3,                    # punctele foarte mici devin vizibile
        line=dict(width=0.5, color="black")  # contur fin
    )
)


# =========================
# GRAFIC 3: Harta - ROI mediu pe tari (mai relevant decat doar count)
# =========================
def extract_countries(val):
    try:
        if isinstance(val, str):
            data = ast.literal_eval(val)
        else:
            data = val
        if isinstance(data, list):
            return [d.get("name") for d in data if isinstance(d, dict) and d.get("name")]
    except Exception:
        return []
    return []

if "production_countries" in df_roi.columns:
    df_roi["country_list"] = df_roi["production_countries"].apply(extract_countries)
    df_countries = df_roi.explode("country_list").rename(columns={"country_list": "country"})

    country_roi = (
        df_countries
        .dropna(subset=["country", "roi"])
        .groupby("country")
        .agg(
            num_titles=("title", "count"),
            avg_roi=("roi", "mean")
        )
        .reset_index()
    )

    # log1p pentru contrast
    country_roi["log_avg_roi"] = np.log1p(country_roi["avg_roi"].clip(lower=0))

    fig3 = px.choropleth(
        country_roi,
        locations="country",
        locationmode="country names",
        color="log_avg_roi",
        hover_name="country",
        hover_data={"num_titles": True, "avg_roi": True, "log_avg_roi": False},
        color_continuous_scale="Plasma",
        title="ROI mediu pe tari (scara logaritmica)"
    )
else:
    fig3 = px.scatter(title="Nu exista coloana production_countries in dataset")

# =========================
# DASH LAYOUT
# =========================
app = Dash(__name__)
app.layout = html.Div([
    html.H1("Raport vizual - Filme (ROI / Rentabilitate)"),

    html.H2("1) Top 10 filme dupa ROI"),
    dcc.Graph(figure=fig1),

    html.H2("3) Rating vs ROI (Popularity ca marime)"),
    dcc.Graph(figure=fig2),

    html.H2("4) ROI mediu pe tari"),
    dcc.Graph(figure=fig3),
])

if __name__ == "__main__":
    app.run(debug=True, port=8050)


C:\Users\pirjo\AppData\Local\Temp\ipykernel_27696\2898243815.py:123: DeprecationWarning:

The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.

